# Exporting and Importing MLRun Functions

In [13]:
# nuclio: ignore
import nuclio

## Write a function

In [14]:
import os
import zipfile

def open_archive(context, 
                 target_dir: str = 'content',
                 archive_url: str = ''):
    """Open a file/object archive into a target directory
    
    :param target_dir   target directory
    :param archive_url  source archive path/url    
    """
        
    # Define locations
    os.makedirs(target_dir, exist_ok=True)
    context.logger.info('Verified directories')
    
    # Extract dataset from zip
    context.logger.info('Extracting zip')
    zip_ref = zipfile.ZipFile(archive_url, 'r')
    zip_ref.extractall(target_dir)
    zip_ref.close()
    
    context.logger.info(f'extracted archive to {target_dir}')
    context.log_artifact('content', target_path=target_dir)


In [15]:
# nuclio: end-code

## Export to a file

In [16]:
import mlrun

In [17]:
# create job function object from notebook code
fn = mlrun.code_to_function('file_utils', runtime='job', with_doc=True,
                            handler=open_archive, image='mlrun/mlrun:latest')
fn.spec.default_handler = 'open_archive'

In [18]:
print(fn.to_yaml())

kind: job
metadata:
  name: file-utils
  tag: ''
  project: ''
spec:
  command: ''
  args: []
  image: mlrun/mlrun:latest
  volumes: []
  volume_mounts: []
  env: []
  default_handler: open_archive
  entry_points:
    open_archive:
      name: open_archive
      doc: Open a file/object archive into a target directory
      parameters:
      - name: context
      - name: target_dir
        type: str
        default: '''content'''
      - name: archive_url
        type: str
        default: ''''''
      outputs: []
      lineno: 6
  description: ''
  build:
    functionSourceCode: IyBHZW5lcmF0ZWQgYnkgbnVjbGlvLmV4cG9ydC5OdWNsaW9FeHBvcnRlciBvbiAyMDIwLTAyLTIxIDEyOjUzCgppbXBvcnQgb3MKaW1wb3J0IHppcGZpbGUKCmRlZiBvcGVuX2FyY2hpdmUoY29udGV4dCwgCiAgICAgICAgICAgICAgICAgdGFyZ2V0X2Rpcjogc3RyID0gJ2NvbnRlbnQnLAogICAgICAgICAgICAgICAgIGFyY2hpdmVfdXJsOiBzdHIgPSAnJyk6CiAgICAiIiJPcGVuIGEgZmlsZS9vYmplY3QgYXJjaGl2ZSBpbnRvIGEgdGFyZ2V0IGRpcmVjdG9yeQogICAgCiAgICA6cGFyYW0gdGFyZ2V0X2RpciAgIHRhcmdldCBkaXJlY3RvcnkKIC

In [19]:
# save to a file (and can be pushed to a git)
fn.export('function.yaml')

[mlrun] 2020-02-21 12:53:22,306 function spec saved to path: function.yaml


## Import the function from Github and run

In [8]:
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

In [20]:
# load from local file
xfn = mlrun.import_function('./function.yaml')

# load function from Github
# xfn = mlrun.import_function('https://raw.githubusercontent.com/mlrun/functions/master/fileutils/open_archive/function.yaml')

# configute it: mount on iguazio fabric, set as interactive (return stdout)
xfn.apply(mlrun.mount_v3io())

# create and run the task
images_path = '/User/mlrun/examples/images'
open_archive_task = mlrun.NewTask('download',  
    params={'target_dir': images_path},
    inputs={'archive_url': 'http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip'})

# run
run = xfn.run(open_archive_task)

[mlrun] 2020-02-21 12:53:52,643 starting run download uid=526b5c30b54e4e02b94030530e418681  -> http://10.233.7.216:8080
[mlrun] 2020-02-21 12:53:52,703 Job is running in the background, pod: download-rbxpt
[mlrun] 2020-02-21 12:53:59,200 downloading http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip to local tmp
[mlrun] 2020-02-21 12:54:00,798 Verified directories
[mlrun] 2020-02-21 12:54:00,798 Extracting zip
[mlrun] 2020-02-21 12:54:07,943 extracted archive to /User/mlrun/examples/images
[mlrun] 2020-02-21 12:54:07,950 log artifact content at /User/mlrun/examples/images, size: None, db: Y

[mlrun] 2020-02-21 12:54:07,959 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...418681,0,Feb 21 12:53:59,completed,download,host=download-rbxptkind=jobowner=admin,archive_url,target_dir=/User/mlrun/examples/images,,content


to track results use .show() or .logs() or in CLI: 
!mlrun get run 526b5c30b54e4e02b94030530e418681  , !mlrun logs 526b5c30b54e4e02b94030530e418681 
[mlrun] 2020-02-21 12:54:10,952 run executed, status=completed
